<a href="https://colab.research.google.com/github/studyy8202/Cleaning_data/blob/main/Cleaning_and_transforming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New section

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('/content/Uncleaned_DS_jobs.csv')

In [5]:
df.shape

(672, 15)

In [3]:
# Check for missing values
missing_values = df.isnull().sum()
print(missing_values)

index                0
Job Title            0
Salary Estimate      0
Job Description      0
Rating               0
Company Name         0
Location             0
Headquarters         0
Size                 0
Founded              0
Type of ownership    0
Industry             0
Sector               0
Revenue              0
Competitors          0
dtype: int64


In [4]:
df.drop_duplicates(inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              672 non-null    int64  
 1   Job Title          672 non-null    object 
 2   Salary Estimate    672 non-null    object 
 3   Job Description    672 non-null    object 
 4   Rating             672 non-null    float64
 5   Company Name       672 non-null    object 
 6   Location           672 non-null    object 
 7   Headquarters       672 non-null    object 
 8   Size               672 non-null    object 
 9   Founded            672 non-null    int64  
 10  Type of ownership  672 non-null    object 
 11  Industry           672 non-null    object 
 12  Sector             672 non-null    object 
 13  Revenue            672 non-null    object 
 14  Competitors        672 non-null    object 
dtypes: float64(1), int64(2), object(12)
memory usage: 78.9+ KB


In [30]:
df.describe()

,Rating,Founded
count,672.000000,672.000000
mean,3.593006,1635.529762
std,1.176684,756.746640
min,0.000000,-1.000000
25%,3.300000,1917.750000
50%,3.800000,1995.000000
75%,4.300000,2009.000000
max,5.000000,2019.000000


In [26]:
df.drop('index', axis=1, inplace=True)

In [28]:
df['Rating'] = df['Rating'].replace(-1, 0)

In [6]:
# Remove unwanted characters from text columns
df['Job Description'] = df['Job Description'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'<.*?>', '', str(x)))).str.strip()

In [31]:
skills = ['Python', 'SQL', 'R', 'AWS', 'Excel', 'Tableau', 'C++', 'Spark']
for skill in skills:
    df[skill] = df['Job Description'].apply(lambda x: 1 if skill.lower() in x.lower() else 0)

In [ ]:
df['Job Description'][0]

In [33]:
df['Size'].unique()

array(['1001 to 5000 employees', '5001 to 10000 employees',
       '501 to 1000 employees', '51 to 200 employees', '10000+ employees',
       '201 to 500 employees', '1 to 50 employees', '-1', 'Unknown'],
      dtype=object)

In [34]:
df['Size'] = df['Size'].replace(-1, 'Unknown')

In [35]:
df['Founded'] = df['Founded'].replace(-1, '2024')

In [36]:
df['Industry'] = df['Industry'].replace(-1, 'Not classified')

In [37]:
df['Revenue'] = df['Revenue'].replace(-1, 'Unknown')

In [32]:
df.head(5)

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,...,Remote,Job Type,Python,SQL,R,AWS,Excel,Tableau,C++,Spark
0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description The Senior Data Scientist is respo...,3.1,Healthfirst.,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,...,No,Unknown,0,0,1,1,0,0,0,0
1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future Join the...",4.2,ManTech.,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,...,No,Unknown,0,1,1,0,0,0,0,0
2,Data Scientist,$137K-$171K (Glassdoor est.),Overview Analysis Group is one of the largest ...,3.8,Analysis Group.,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,...,No,Unknown,1,0,1,1,1,0,1,0
3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION: Do you have a passion for Dat...,3.5,INFICON.,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,...,No,Unknown,1,1,1,1,1,0,0,0
4,Data Scientist,$137K-$171K (Glassdoor est.),Data Scientist Affinity Solutions / Marketing ...,2.9,Affinity Solutions.,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,...,No,Unknown,1,1,1,0,1,0,0,0


In [7]:
# Remove numbers from company_name
df['Company Name'] = df['Company Name'].apply(lambda x: re.sub(r'\d+', '', str(x))).str.strip()


In [8]:
# Remove newline characters from company_name
df['Company Name'] = df['Company Name'].apply(lambda x: x.replace('\n', '').strip())


In [9]:
# Function to extract city and state from location
def extract_city_state(location):
    if 'Remote' in location:  # Check if the location is "Remote"
        return pd.Series([None, None])
    else:
        parts = location.split(',')
        if len(parts) == 1:  # Only state is provided
            return pd.Series([None, parts[0].strip()])
        elif len(parts) == 2:  # Both city and state are provided
            return pd.Series([parts[0].strip(), parts[1].strip()])
        else:  # Any other unexpected format (should not happen in your case)
            return pd.Series([None, None])

In [10]:
# Apply the function to the location column
df[['City', 'State']] = df['Location'].apply(extract_city_state)


In [11]:
#Add a column to indicate if the job is remote
df['Remote'] = df['Location'].apply(lambda x: 'Yes' if 'Remote' in x else 'No')

In [12]:
df['Competitors'] = df['Competitors'].replace("-1", np.nan)

In [13]:
# Identify job type based on description
df['Job Type'] = df['Job Description'].apply(lambda x: 'Full-time' if 'full-time' in x.lower() else
                                                      'Part-time' if 'part-time' in x.lower() else
                                                       'Unknown')

# EDA

In [15]:
top_10_job_titiles = df['Job Title'].value_counts().head(10)
print(top_10_job_titiles)

Job Title
Data Scientist                                                                                      337
Data Engineer                                                                                        26
Senior Data Scientist                                                                                19
Machine Learning Engineer                                                                            16
Data Analyst                                                                                         12
Senior Data Analyst                                                                                   6
Senior Data Engineer                                                                                  5
Data Science Software Engineer                                                                        4
ENGINEER - COMPUTER SCIENTIST - RESEARCH COMPUTER SCIENTIST - SIGNAL PROCESSING - SAN ANTONIO OR      4
Data Scientist - TS/SCI FSP or CI Required            

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
#Filter the Data for "Data Scientist" job titles
data_scientist_jobs = df[df['Job Title'].str.lower() == 'data scientist']

In [17]:
# Step 2: Tokenize and remove stopwords from job descriptions
vectorizer = CountVectorizer(stop_words='english')

In [18]:
# Fit and transform the job descriptions for Data Scientist roles
X = vectorizer.fit_transform(data_scientist_jobs['Job Description'])

In [19]:
# Create a DataFrame of word counts
word_counts = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())


In [20]:
# Step 3: Sum up the word counts and find the most common skill
most_common_skills = word_counts.sum().sort_values(ascending=False)


In [23]:
# Print the top skill
print(most_common_skills.head(30))

data           3776
experience     1632
learning        756
science         756
work            694
business        662
machine         607
analysis        573
team            547
skills          524
scientist       480
models          447
analytics       437
ability         430
years           421
using           411
solutions       393
statistical     381
tools           377
engineering     367
company         360
python          352
systems         337
job             337
information     334
development     333
working         331
knowledge       323
technical       320
techniques      312
dtype: int64
